In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
import random
InteractiveShell.ast_node_interactivity = "all"
import os
from datetime import datetime
import matplotlib.pyplot as plt
#import plotly.express as px
import glob
from tqdm import tqdm
from sklearn import preprocessing
import sweetviz as sv

In [2]:
# import data
url = '../data/viirs-snpp_2021_Australia.csv'
fileName = '../data/viirs-snpp_2021_Australia'
aus_fires = pd.read_csv(url, parse_dates=['acq_date'])
aus_fires.shape
aus_fires.head()
aus_fires.drop_duplicates()
aus_fires.info()

(812052, 15)

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type
0,-37.868965,145.241455,330.10,0.47,0.40,2021-01-01,354,N,VIIRS,l,1,303.56,2.36,D,0
1,-35.096336,138.580673,341.55,0.63,0.54,2021-01-01,354,N,VIIRS,n,1,301.13,3.57,D,0
2,-28.734032,148.792267,355.01,0.41,0.37,2021-01-01,354,N,VIIRS,n,1,313.61,24.69,D,0
3,-25.566595,146.441620,356.92,0.39,0.36,2021-01-01,354,N,VIIRS,n,1,314.58,10.19,D,0
4,-26.489180,148.390457,340.80,0.41,0.37,2021-01-01,354,N,VIIRS,n,1,310.51,4.52,D,0


,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type
0,-37.868965,145.241455,330.10,0.47,0.40,2021-01-01,354,N,VIIRS,l,1,303.56,2.36,D,0
1,-35.096336,138.580673,341.55,0.63,0.54,2021-01-01,354,N,VIIRS,n,1,301.13,3.57,D,0
2,-28.734032,148.792267,355.01,0.41,0.37,2021-01-01,354,N,VIIRS,n,1,313.61,24.69,D,0
3,-25.566595,146.441620,356.92,0.39,0.36,2021-01-01,354,N,VIIRS,n,1,314.58,10.19,D,0
4,-26.489180,148.390457,340.80,0.41,0.37,2021-01-01,354,N,VIIRS,n,1,310.51,4.52,D,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
812047,-20.612724,116.773460,310.05,0.44,0.63,2021-12-31,1806,N,VIIRS,n,1,295.76,2.19,N,2
812048,-21.697954,115.002548,305.72,0.35,0.57,2021-12-31,1806,N,VIIRS,n,1,294.84,0.77,N,2
812049,-20.597576,116.783463,331.56,0.44,0.63,2021-12-31,1806,N,VIIRS,n,1,295.79,2.24,N,2
812050,-20.821804,115.386932,305.26,0.36,0.58,2021-12-31,1806,N,VIIRS,n,1,294.65,0.43,N,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 812052 entries, 0 to 812051
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   latitude    812052 non-null  float64       
 1   longitude   812052 non-null  float64       
 2   bright_ti4  812052 non-null  float64       
 3   scan        812052 non-null  float64       
 4   track       812052 non-null  float64       
 5   acq_date    812052 non-null  datetime64[ns]
 6   acq_time    812052 non-null  int64         
 7   satellite   812052 non-null  object        
 8   instrument  812052 non-null  object        
 9   confidence  812052 non-null  object        
 10  version     812052 non-null  int64         
 11  bright_ti5  812052 non-null  float64       
 12  frp         812052 non-null  float64       
 13  daynight    812052 non-null  object        
 14  type        812052 non-null  int64         
dtypes: datetime64[ns](1), float64(7), int64(3), object(

In [3]:
aus_fires['latitude'] = pd.to_numeric(aus_fires['latitude'], errors='coerce')
aus_fires['longitude'] = pd.to_numeric(aus_fires['longitude'], errors='coerce')
aus_fires['bright_ti4'] = pd.to_numeric(aus_fires['bright_ti4'], errors='coerce')
aus_fires['acq_date'] = pd.to_datetime(aus_fires['acq_date'], errors='coerce')
aus_fires['scan'] = pd.to_numeric(aus_fires['scan'], errors='coerce')
aus_fires['track'] = pd.to_numeric(aus_fires['track'], errors='coerce')
aus_fires['bright_ti5'] = pd.to_numeric(aus_fires['bright_ti5'], errors='coerce')
aus_fires['frp'] = pd.to_numeric(aus_fires['frp'], errors='coerce')


In [4]:
aus_fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 812052 entries, 0 to 812051
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   latitude    812052 non-null  float64       
 1   longitude   812052 non-null  float64       
 2   bright_ti4  812052 non-null  float64       
 3   scan        812052 non-null  float64       
 4   track       812052 non-null  float64       
 5   acq_date    812052 non-null  datetime64[ns]
 6   acq_time    812052 non-null  int64         
 7   satellite   812052 non-null  object        
 8   instrument  812052 non-null  object        
 9   confidence  812052 non-null  object        
 10  version     812052 non-null  int64         
 11  bright_ti5  812052 non-null  float64       
 12  frp         812052 non-null  float64       
 13  daynight    812052 non-null  object        
 14  type        812052 non-null  int64         
dtypes: datetime64[ns](1), float64(7), int64(3), object(

In [5]:
aus_fires.rename({'acq_date': 'time'}, axis=1, inplace=True)
aus_fires['est_fire_area'] = aus_fires['scan'] * aus_fires['track']
aus_fires['est_brightness'] = (aus_fires['bright_ti4'] + aus_fires['bright_ti5'])/2
aus_fires.latitude = aus_fires.latitude.round(1)
aus_fires.longitude = aus_fires.longitude.round(1)
aus_fires = aus_fires[aus_fires.type==0]

aus_fires.shape
aus_fires.head()

(795915, 17)

,latitude,longitude,bright_ti4,scan,track,time,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight,type,est_fire_area,est_brightness
0,-37.9,145.2,330.10,0.47,0.40,2021-01-01,354,N,VIIRS,l,1,303.56,2.36,D,0,0.1880,316.830
1,-35.1,138.6,341.55,0.63,0.54,2021-01-01,354,N,VIIRS,n,1,301.13,3.57,D,0,0.3402,321.340
2,-28.7,148.8,355.01,0.41,0.37,2021-01-01,354,N,VIIRS,n,1,313.61,24.69,D,0,0.1517,334.310
3,-25.6,146.4,356.92,0.39,0.36,2021-01-01,354,N,VIIRS,n,1,314.58,10.19,D,0,0.1404,335.750
4,-26.5,148.4,340.80,0.41,0.37,2021-01-01,354,N,VIIRS,n,1,310.51,4.52,D,0,0.1517,325.655


In [23]:
fires = aus_fires[['latitude', 'longitude', 'time','confidence',
                           'est_fire_area','est_brightness','frp']].copy()
## Add fire_count column
count = fires.groupby(['latitude', 'longitude', 'time']).size().reset_index().rename(columns={0:'fire_count'})
fire_copy = fires.merge(count,how='outer', on=['latitude', 'longitude', 'time'])

fire_copy

,latitude,longitude,time,confidence,est_fire_area,est_brightness,frp,fire_count
0,-37.9,145.2,2021-01-01,l,0.1880,316.830,2.36,1
1,-35.1,138.6,2021-01-01,n,0.3402,321.340,3.57,1
2,-28.7,148.8,2021-01-01,n,0.1517,334.310,24.69,2
3,-28.7,148.8,2021-01-01,n,0.1404,299.095,1.18,2
4,-25.6,146.4,2021-01-01,n,0.1404,335.750,10.19,2
...,...,...,...,...,...,...,...,...
795910,-29.0,127.8,2021-12-31,n,0.1404,310.270,1.52,5
795911,-29.0,127.8,2021-12-31,n,0.1404,298.305,1.52,5
795912,-29.0,127.8,2021-12-31,n,0.1404,297.760,2.18,5
795913,-29.0,127.8,2021-12-31,n,0.1404,320.180,2.18,5


In [24]:
fire_copy.confidence = fire_copy.confidence.replace({'l': 20, 'n': 60, 'h': 100})

In [25]:
fire_copy2 = fire_copy.groupby(['latitude', 'longitude', 'time'])[['fire_count','confidence','est_brightness']].mean().reset_index()
fire_copy2

,latitude,longitude,time,fire_count,confidence,est_brightness
0,-43.4,146.9,2021-04-27,34.0,52.941176,314.065588
1,-43.4,147.0,2021-04-27,18.0,55.555556,305.837778
2,-43.3,146.9,2021-05-01,11.0,63.636364,328.005000
3,-43.3,147.0,2021-05-01,29.0,58.620690,311.472069
4,-43.3,147.3,2021-04-30,1.0,60.000000,308.205000
...,...,...,...,...,...,...
117326,-10.1,142.1,2021-10-22,1.0,20.000000,319.965000
117327,-10.1,142.2,2021-09-08,1.0,60.000000,312.640000
117328,-10.1,142.2,2021-09-22,10.0,64.000000,316.170500
117329,-10.1,142.2,2021-10-22,1.0,20.000000,318.450000


In [30]:
fire_copy3 = fire_copy.groupby(['latitude', 'longitude', 'time'])[['frp','est_fire_area']].sum().reset_index()
fire_copy3

,latitude,longitude,time,frp,est_fire_area
0,-43.4,146.9,2021-04-27,309.91,5.1440
1,-43.4,147.0,2021-04-27,111.33,3.2536
2,-43.3,146.9,2021-05-01,2736.29,2.5023
3,-43.3,147.0,2021-05-01,3294.38,6.7963
4,-43.3,147.3,2021-04-30,2.27,0.1904
...,...,...,...,...,...
117326,-10.1,142.1,2021-10-22,5.41,0.1794
117327,-10.1,142.2,2021-09-08,4.13,0.1794
117328,-10.1,142.2,2021-09-22,79.41,4.4268
117329,-10.1,142.2,2021-10-22,4.39,0.1794


In [29]:
fire_copy

,latitude,longitude,time,confidence,est_fire_area,est_brightness,frp,fire_count
0,-37.9,145.2,2021-01-01,20,0.1880,316.830,2.36,1
1,-35.1,138.6,2021-01-01,60,0.3402,321.340,3.57,1
2,-28.7,148.8,2021-01-01,60,0.1517,334.310,24.69,2
3,-28.7,148.8,2021-01-01,60,0.1404,299.095,1.18,2
4,-25.6,146.4,2021-01-01,60,0.1404,335.750,10.19,2
...,...,...,...,...,...,...,...,...
795910,-29.0,127.8,2021-12-31,60,0.1404,310.270,1.52,5
795911,-29.0,127.8,2021-12-31,60,0.1404,298.305,1.52,5
795912,-29.0,127.8,2021-12-31,60,0.1404,297.760,2.18,5
795913,-29.0,127.8,2021-12-31,60,0.1404,320.180,2.18,5


In [31]:
fire_copy = fire_copy2.merge(fire_copy3,how='outer', on=['latitude', 'longitude', 'time'])


In [32]:
fire_copy

,latitude,longitude,time,fire_count,confidence,est_brightness,frp,est_fire_area
0,-43.4,146.9,2021-04-27,34.0,52.941176,314.065588,309.91,5.1440
1,-43.4,147.0,2021-04-27,18.0,55.555556,305.837778,111.33,3.2536
2,-43.3,146.9,2021-05-01,11.0,63.636364,328.005000,2736.29,2.5023
3,-43.3,147.0,2021-05-01,29.0,58.620690,311.472069,3294.38,6.7963
4,-43.3,147.3,2021-04-30,1.0,60.000000,308.205000,2.27,0.1904
...,...,...,...,...,...,...,...,...
117326,-10.1,142.1,2021-10-22,1.0,20.000000,319.965000,5.41,0.1794
117327,-10.1,142.2,2021-09-08,1.0,60.000000,312.640000,4.13,0.1794
117328,-10.1,142.2,2021-09-22,10.0,64.000000,316.170500,79.41,4.4268
117329,-10.1,142.2,2021-10-22,1.0,20.000000,318.450000,4.39,0.1794


In [33]:
## Add location count
count = fire_copy.groupby(['latitude', 'longitude']).size().reset_index().rename(columns={0:'loc_count'})
fire_copy = fire_copy.merge(count,how='outer', on=['latitude', 'longitude'])

fire_copy

,latitude,longitude,time,fire_count,confidence,est_brightness,frp,est_fire_area,loc_count
0,-43.4,146.9,2021-04-27,34.0,52.941176,314.065588,309.91,5.1440,1
1,-43.4,147.0,2021-04-27,18.0,55.555556,305.837778,111.33,3.2536,1
2,-43.3,146.9,2021-05-01,11.0,63.636364,328.005000,2736.29,2.5023,1
3,-43.3,147.0,2021-05-01,29.0,58.620690,311.472069,3294.38,6.7963,1
4,-43.3,147.3,2021-04-30,1.0,60.000000,308.205000,2.27,0.1904,1
...,...,...,...,...,...,...,...,...,...
117326,-10.1,142.1,2021-10-22,1.0,20.000000,319.965000,5.41,0.1794,3
117327,-10.1,142.2,2021-09-08,1.0,60.000000,312.640000,4.13,0.1794,3
117328,-10.1,142.2,2021-09-22,10.0,64.000000,316.170500,79.41,4.4268,3
117329,-10.1,142.2,2021-10-22,1.0,20.000000,318.450000,4.39,0.1794,3


In [34]:
fire_copy.est_fire_area = fire_copy.est_fire_area.round(1)
fire_copy.est_brightness = fire_copy.est_brightness.round(1)
fire_copy.confidence = fire_copy.confidence.round().astype(int)
fire_copy.frp = fire_copy.frp.round(1)
fire_copy.fire_count = fire_copy.fire_count.round().astype(int)

fire_copy.head(20)

,latitude,longitude,time,fire_count,confidence,est_brightness,frp,est_fire_area,loc_count
0,-43.4,146.9,2021-04-27,34,53,314.1,309.9,5.1,1
1,-43.4,147.0,2021-04-27,18,56,305.8,111.3,3.3,1
2,-43.3,146.9,2021-05-01,11,64,328.0,2736.3,2.5,1
3,-43.3,147.0,2021-05-01,29,59,311.5,3294.4,6.8,1
4,-43.3,147.3,2021-04-30,1,60,308.2,2.3,0.2,1
5,-43.2,146.8,2021-05-12,1,60,301.6,6.6,0.2,1
6,-43.2,146.9,2021-03-06,6,60,308.9,131.1,1.0,1
7,-43.2,147.1,2021-04-09,1,60,309.0,7.3,0.3,4
8,-43.2,147.1,2021-06-04,1,60,286.1,1.8,0.2,4
9,-43.2,147.1,2021-06-05,1,60,287.1,0.6,0.1,4


In [35]:
confidence_norm = preprocessing.normalize([fire_copy['confidence']], norm='max')
frp_norm = preprocessing.normalize([fire_copy['frp']], norm='max')
area_norm = preprocessing.normalize([fire_copy['est_fire_area']], norm='max')
count_norm = preprocessing.normalize([fire_copy['fire_count']], norm='max')

In [36]:
fire_copy['ranking'] = confidence_norm[0]*0.4 + frp_norm[0]*0.2 + area_norm[0]*0.2 + count_norm[0]*0.2

In [40]:
fire_copy = fire_copy.sort_values(by=['ranking'], ascending=False)
fire_copy

,latitude,longitude,time,fire_count,confidence,est_brightness,frp,est_fire_area,loc_count,ranking
20902,-31.6,123.8,2021-12-17,402,72,320.6,10445.8,123.2,8,0.839755
3115,-37.2,141.1,2021-12-31,280,80,332.6,13766.7,51.3,3,0.742583
69531,-16.3,137.5,2021-12-10,248,68,323.0,6956.2,106.8,12,0.669818
25798,-29.3,129.2,2021-12-18,253,71,323.0,7722.6,77.8,5,0.648362
46652,-20.9,145.4,2021-10-23,313,63,298.0,2545.5,119.4,4,0.638533
...,...,...,...,...,...,...,...,...,...,...
87138,-14.6,133.3,2021-06-29,1,20,319.5,0.8,0.1,14,0.080671
51574,-19.2,146.6,2021-11-07,1,20,318.3,0.8,0.1,5,0.080671
104748,-12.9,134.7,2021-07-26,1,20,317.7,0.8,0.1,18,0.080671
106584,-12.7,132.0,2021-05-28,1,20,318.9,0.7,0.1,12,0.080670


In [38]:
import sweetviz as sv
analyze_report = sv.analyze(fire_copy)
analyze_report.show_html(f'{fileName}.html', open_browser=False)


Report ../data/viirs-snpp_2021_Australia.html was generated.


In [39]:
fire_copy.to_csv(f"{fileName}_prepared.csv.gz", index=False, compression='gzip')
fire_copy.to_csv(f"{fileName}_prepared.csv", index=False)